In [15]:
import os
from datetime import datetime
from IPython.display import display
from ipywidgets import interact, interact_manual
import ipywidgets as widgets
from study_common import *
from study_notebook_helper import *

study_datasets = os.listdir(STUDY_RESULTS_PATH)
@interact(dataset=[None] + study_datasets)
def main(name="erik", dataset=None):
    if len(name) == 0:
        print("Please choose a name / id for the study")
        return
    if dataset is None:
        print("Hey " + name + ", please chose a dataset")
        return
    print(name + " is doing " + str(dataset) + ("\n" * 10))
    study: STUDY_TYPE = None
    with open(STUDY_RESULTS_PATH + dataset, "rb") as data_file:
        study = pickle.load(data_file)
    random.Random(name).shuffle(study[1])  # shuffle study entries seeded by participant name
    r = LocalRepo(study[0])
    print_html("<h1>Study Time!</h1>")
    print("\n" * 10)
    study_entry = study[1][0]
    pattern = study_entry[0]
    m0 = study_entry[1]
    m1 = study_entry[2]
    commits = study_entry[3]
    commit_cells: List[str] = []
    commit_empty_cell = '<td style="border: 1px solid black; background-color: #aaa;"></td>'
    for c in commits:
        # 0 message, 1 author, 2 date, 3 hexsha, 4 [0 files, 1 additions, 2 deletions], 5 belongs to method 1, 6 belongs to method 2
        datestring = datetime.utcfromtimestamp(c[2]).strftime('%Y-%m-%d')
        commit_cells.append(f"""<tr>
            {commit_empty_cell if not c[5] else ""}
            <td{' colspan="2"' if c[5] and c[6] else ""} style="border: 1px solid black;"><center>
                {c[1]}: <a href="https://github.com/{r.name}/commit/{c[3]}" target="_blank">{c[0]}</a><br>
                <span style="font-family: monospace">
                    {"{:,}".format(c[4][0])} changed files:
                    <span style="color:green">+{"{:,}".format(c[4][1])}</span>
                    <span style="color:red">-{"{:,}".format(c[4][2])}</span>,
                    created on {datestring}
                </span>
            </center></td>
            {commit_empty_cell if not c[6] else ""}
            </tr>""")
    
    print_html(f"""{PRISM_HIGHLIGHTING}{OWN_STYLE}
You will be shown multiple methods from the <a href="{r.url()}" target="_blank" style="color: blue">{study[0]}</a> repository.
Take a close look at the two presented snippets of code.
Consider everything, not only the statements, but also variable naming, comments,
location of this snippet within the project, editing history, and relations with other code.
Then, please select how closely the presented phrases describe this pair of code pieces.
<br><br><br><br>
<table border="1">
  <tr>
    <td>{path_html(r, m0[0])}</td> 
    <td>{path_html(r, m1[0])}</td>
  </tr>
  <tr>
    <td><pre><code class="language-java">{m0[1]}</code></pre></td>
    <td><pre><code class="language-java">{m1[1]}</code></pre></td>
  </tr>
  <tr><td colspan="2"><center>And here are the commits: {len(commits)}</center></td></tr>
  {"".join(commit_cells)}
</table>

    """)

interactive(children=(Text(value='erik', description='name'), Dropdown(description='dataset', options=(None, '…